In [1]:
from IPython.display import display, Markdown

In [27]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [26]:
%%bash
pip install langchain-community beautifulsoup4 langchain-chroma langchain-openai

  Using cached langchain_openai-0.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached langchain_openai-0.2.2-py3-none-any.whl (49 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.6/982.6 kB 8.3 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)


In [3]:
from langchain_community.document_loaders import RecursiveUrlLoader

loader = RecursiveUrlLoader(
    "https://www.scouts.org.uk/por/",
    max_depth=5,
    # use_async=False,
    extractor=None,
    # metadata_extractor=None,
    # exclude_dirs=(),
    # timeout=10,
    check_response_status=True,
    # continue_on_failure=True,
    # prevent_outside=True,
    # base_url=None,
    # ...
)

In [4]:
docs = loader.load()

Unable to load from https://www.scouts.org.uk/por/definitions-chapter/03453001818. Received error Received HTTP status 404 of type ValueError
Unable to load from https://www.scouts.org.uk/por/15-resolving-concerns/03453001818. Received error Received HTTP status 404 of type ValueError
Unable to load from https://www.scouts.org.uk/por/12-flags-and-ceremonial/03453001818. Received error Received HTTP status 404 of type ValueError
Unable to load from https://www.scouts.org.uk/por/03453001818. Received error Received HTTP status 404 of type ValueError


In [5]:
print(docs[0].page_content)

<!DOCTYPE html><html lang=en prefix="og: http://ogp.me/ns#" data-ssr><head><!-- Google Tag Manager - initialisation script placeholder (/src/services/gtm.js -> setupGTM()) --><script id=gtm-script></script><title>Policy, Organisation and Rules | Scouts</title><link rel=apple-touch-icon sizes=180x180 href=/apple-touch-icon.png><!--[if IE]><link rel="icon" type="image/png" sizes="32x32" href="/favicon-32x32.png"><![endif]--><link rel=icon type=image/png sizes=16x16 href=/favicon-16x16.png><link rel=manifest href=/site.webmanifest><meta name=referrer content=no-referrer><meta name=msapplication-TileColor content=#ffffff><meta name=theme-color content=#ffffff><meta data-vm="ssr" charset="utf-8"><meta data-vm="ssr" name="viewport" content="width=device-width, initial-scale=1"><meta data-vm="ssr" http-equiv="X-UA-Compatible" content="IE=edge"><meta data-vm="ssr" name="mobile-web-app-capable" content="yes"><meta data-vm="ssr" data-vmid="og:title" property="og:title" content="Scouts"><meta dat

In [6]:
[d.metadata["source"] for d in docs]

['https://www.scouts.org.uk/por/',
 'https://www.scouts.org.uk/por/12-flags-and-ceremonial/',
 'https://www.scouts.org.uk/por/definitions-chapter/',
 'https://www.scouts.org.uk/por/15-resolving-concerns/',
 'https://www.scouts.org.uk/por/16-adult-roles/',
 'https://www.scouts.org.uk/por/13-trusteeship-property-and-equipment/',
 'https://www.scouts.org.uk/por/7-emergency-procedures/',
 'https://www.scouts.org.uk/por/9-activities/',
 'https://www.scouts.org.uk/por/4-local-organisation-of-sections-groups-districts-counties/',
 'https://www.scouts.org.uk/por/6-the-structure-of-the-uk-headquarters-of-the-scout-association/',
 'https://www.scouts.org.uk/por/3-membership/',
 'https://www.scouts.org.uk/por/2-key-policies/',
 'https://www.scouts.org.uk/por/10-uniform-badges-and-emblems/',
 'https://www.scouts.org.uk/por/11-awards-and-recognition-of-service/',
 'https://www.scouts.org.uk/por/5-local-governance-and-finance-of-groups-districts-counties/',
 'https://www.scouts.org.uk/por/1-our-fund

In [7]:
from langchain_community.document_transformers import Html2TextTransformer

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

In [24]:
from langchain_text_splitters import HTMLSectionSplitter


headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("strong", "Strong")
]

doc = docs[2]
print(doc.metadata)
html_splitter = HTMLSectionSplitter(headers_to_split_on)
html_header_splits = html_splitter.split_text(doc.page_content)
html_header_splits

{'source': 'https://www.scouts.org.uk/por/definitions-chapter/', 'content_type': 'text/html; charset=utf-8', 'title': 'Definitions Chapter | Policy, Organisation and Rules | Scouts', 'description': 'Policy, Organisation and Rules', 'language': 'en'}


[Document(metadata={'Header 1': '#TITLE#'}, page_content="Skip to main content \n         \n \n Volunteering at Scouts is changing to help us reach more young people \n Volunteering is changing to help us reach more young people \n Volunteering is changing at Scouts.  Read more \n Discover what this means \n \n   \n \n Join Scouts \n \n Squirrels (4-6 years) \n Play, learn and get outdoors \n \n Beavers (6-8 years) \n Try new things, make friends \n \n Cubs (8-10½ years) \n Master new skills, have adventures \n \n Scouts (10½-14 years) \n Explore the world, challenge yourself \n \n Explorers (14-18 years) \n Take the lead, embrace change \n \n Network (18-25 years) \n Be the best you can be \n \n Volunteer \n Share your skills, gain new ones \n \n Donate \n \n Help us reach more young people \n \n \n \n \n Info for parents \n \n What do Scouts do? \n Gain skills for life \n \n Keeping young people safe \n Safety and safeguarding \n \n All about badges \n Earn them and show them off \n 

In [28]:
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import ParentDocumentRetriever

In [29]:
# This text splitter is used to create the child documents
# TODO needs to be a TextSplitter
child_splitter = HTMLSectionSplitter(headers_to_split_on)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", 
    embedding_function=OpenAIEmbeddings()
)
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

ValidationError: 1 validation error for ParentDocumentRetriever
child_splitter
  Input should be an instance of TextSplitter [type=is_instance_of, input_value=<langchain_text_splitters...r object at 0x14ea6c290>, input_type=HTMLSectionSplitter]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of

In [ ]:
retriever.add_documents(docs, ids=None)